# DNAmTL

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'dnamtl'
data_type = 'methylation'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2019
approved_by_author = '⌛'
citation = "Lu, Ake T., et al. \"DNA methylation-based estimator of telomere length.\" Aging (Albany NY) 11.16 (2019): 5895."
doi = 'https://doi.org/10.18632/aging.102173'
notes = None

## Download necessary data

#### Download directly with curl

In [3]:
supplementary_url = "https://www.aging-us.com/article/102173/supplementary/SD7/0/aging-v11i16-102173-supplementary-material-SD7.xlsx"
supplementary_file_name = "coefficients.xlsx"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

## Load data

#### From Excel file

In [4]:
df = pd.read_excel('coefficients.xlsx', skiprows=5)

## Extract features and weights

First, let's extract the features and weights:

In [5]:
df['feature'] = df['Variable']
df['coefficient'] = df['Coefficient']

df.head()

,Variable,Coefficient,CHR,bp19,SourceSeq,Strand,Probe SNP\nbased on Illumina annotation,Gene,location,Relation to \nCpGIsland,...,ALLELE,BSGS.P,BSGS.EFFECT,BSGS.H2,LBC.P,LBC.EFFECT,LBC.R2,mQTL,feature,coefficient
0,Intercept,7.924780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Intercept,7.924780
1,cg05528516,-0.034902,1,1070040.0,CGGACCCCCACCGGCCTCCAAATGTGCAAACACAGGCGCCTCTCAG...,R,NaN,C1orf159-TTLL10,NaN,N_Shelf,...,T,2.613000e-42,0.258,0.38864,2.561000e-47,0.1696,0.1420,cis,cg05528516,-0.034902
2,cg00060374,0.192655,1,1355235.0,CGGCTTCATGCTGGTGGCAGCAACAGACTCTCCGCCAGCGCCGGGC...,R,NaN,LOC441869,Body;Body,Island,...,A,2.982000e-58,-0.440,0.45741,9.873000e-84,-0.5123,0.2409,cis,cg00060374,0.192655
3,cg12711627,0.178270,1,2025769.0,CGGGCACCACACAGCATCCCAGGCACCATCATGGTAGGAGAAGAGT...,R,NaN,PRKCZ,Body;5'UTR,Island,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cg12711627,0.178270
4,cg06853416,0.054884,1,2778841.0,AGAGTCCCCCTCTGGATTCACACACCTGGAGGCGTCTGAGTGACTC...,R,NaN,MMEL1-ACTRT2,NaN,S_Shore,...,C,2.594000e-39,-0.206,0.34692,3.236000e-78,-0.2415,0.2267,cis,cg06853416,0.054884


Then, let's create lists for features and weights. Be careful about the intercept, as it usually shows up as a feature name.

In [6]:
features = df['feature'][1:].tolist()
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0).float()
intercept = torch.tensor([df['coefficient'][0]]).float()

## Load weights into pyaging model

#### Linear model

In [7]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=140, out_features=1, bias=True)
)

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [8]:
reference_feature_values = None

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [9]:
preprocessing = None

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [10]:
postprocessing = None

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [11]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [12]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [13]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [14]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [15]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [16]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [17]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: coefficients.xlsx
Deleted folder: .ipynb_checkpoints
